### Comparison of different MFI algortims to indentify the fastest one

#### First, define constants, base functions and load positions.

In [2]:
from traceback import print_tb
import numpy as np
import math
from scipy import stats
import os
import time
import glob
import matplotlib.pyplot as plt
from numba import jit, njit
from numba.typed import List
from matplotlib import ticker

In [3]:
min_grid=np.array((-np.pi, -np.pi))
max_grid=np.array((np.pi, np.pi))
nbins=np.array((200, 200))

gridx = np.linspace(min_grid[0], max_grid[0], nbins[0])
gridy = np.linspace(min_grid[1], max_grid[1], nbins[1])
grid_space = np.array(((max_grid[0] - min_grid[0]) / (nbins[0]-1), (max_grid[1] - min_grid[1]) / (nbins[1]-1)))
X, Y = np.meshgrid(gridx, gridy)

bw = 0.05
bw2 = bw**2
stride = 10
const = (1 / (bw * np.sqrt(2 * np.pi) * stride))
kT = 2.49

In [46]:
def load_HILLS_2D(hills_name="../data/HILLS"):
	"""Load 2-dimensional hills data (includes time, position_x, position_y, hills_parameters ).
	
	Args:
		hills_name (str, optional): Name of hills file. Defaults to "HILLS".
	Returns:
		np.array: Array with hills data
	"""
	for file in glob.glob(hills_name):
		hills = np.loadtxt(file)
		hills = np.concatenate(([hills[0]], hills[:-1]))
		hills[0][5] = 0
	return hills

def load_position_2D(position_name="../data/position"):
	"""Load 2-dimensional position/trajectory data.

	Args:
		position_name (str, optional): Name of position file. Defaults to "position".

	Returns:
		list: 2 * np.array with position data of each dimension ([position_x, position_y])
	"""
	for file1 in glob.glob(position_name):
		colvar = np.loadtxt(file1)
		position_x = colvar[:-1, 1]
		position_y = colvar[:-1, 2]
	return [position_x, position_y]

def find_periodic_point(x_coord, y_coord, min_grid, max_grid, periodic):
    
	"""Finds periodic copies of input coordinates. 
	
	Args:
		x_coord (float): CV1-coordinate
		y_coord (float): CV2-coordinate
		min_grid (list): list of CV1-minimum value of grid and CV2-minimum value of grid
		max_grid (list): list of CV1-maximum value of grid and CV2-maximum value of grid
		periodic (binary): information if system is periodic. value of 0 corresponds to non-periodic system; function will only return input coordinates. Value of 1 corresponds to periodic system; function will return input coordinates with periodic copies.
	Returns:
		list: list of [x-coord, y-coord] pairs
	"""

	coord_list = []
	coord_list.append([x_coord, y_coord])
	
	if periodic == 1:
		# Use periodic extension for defining PBC
		periodic_extension = periodic * 1 / 2
		grid_ext = (1 / 2) * periodic_extension * (max_grid - min_grid)

		# There are potentially 4 points, 1 original and 3 periodic copies, or less.

		copy_record = [0, 0, 0, 0]
		# check for x-copy
		if x_coord < min_grid[0] + grid_ext[0]:
			coord_list.append([x_coord + 2 * np.pi, y_coord])
			copy_record[0] = 1
		elif x_coord > max_grid[0] - grid_ext[0]:
			coord_list.append([x_coord - 2 * np.pi, y_coord])
			copy_record[1] = 1
		# check for y-copy
		if y_coord < min_grid[1] + grid_ext[1]:
			coord_list.append([x_coord, y_coord + 2 * np.pi])
			copy_record[2] = 1
		elif y_coord > max_grid[1] - grid_ext[1]:
			coord_list.append([x_coord, y_coord - 2 * np.pi])
			copy_record[3] = 1
		# check for xy-copy
		if sum(copy_record) == 2:
			if copy_record[0] == 1 and copy_record[2] == 1:
				coord_list.append([x_coord + 2 * np.pi, y_coord + 2 * np.pi])
			elif copy_record[1] == 1 and copy_record[2] == 1:
				coord_list.append([x_coord - 2 * np.pi, y_coord + 2 * np.pi])
			elif copy_record[0] == 1 and copy_record[3] == 1:
				coord_list.append([x_coord + 2 * np.pi, y_coord - 2 * np.pi])
			elif copy_record[1] == 1 and copy_record[3] == 1:
				coord_list.append([x_coord - 2 * np.pi, y_coord - 2 * np.pi])        

	return coord_list

def mean_force_variance(Ftot_den, Ftot_den2, Ftot_x, Ftot_y, ofv_num_x, ofv_num_y, Ftot_den_limit=10**-10):

	bessel_corr = np.divide(Ftot_den**2 , (Ftot_den**2-Ftot_den2), out=np.zeros_like(Ftot_den), where=(Ftot_den**2-Ftot_den2) > 0)
		
	ofv_x = (np.divide(ofv_num_x , Ftot_den, out=np.zeros_like(Ftot_den), where=Ftot_den > Ftot_den_limit) - Ftot_x**2) * bessel_corr
	ofe_x = np.sqrt(ofv_x)
	
	ofv_y = (np.divide(ofv_num_y , Ftot_den, out=np.zeros_like(Ftot_den), where=Ftot_den > Ftot_den_limit) - Ftot_y**2) * bessel_corr
	ofe_y = np.sqrt(ofv_y)
	
	ofv = np.sqrt(ofv_x**2 + ofv_y**2)
	ofe = np.sqrt(ofe_x**2 + ofe_y**2)
		
	return [ofv, ofe]

def FFT_intg_2D(FX, FY, min_grid=np.array((-np.pi, -np.pi)), max_grid=np.array((np.pi, np.pi)), nbins=0):
	"""2D integration of force gradient (FX, FY) to find FES using Fast Fourier Transform.

	Args:
		FX (array of size (nbins[0], nbins[1])): CV1 component of the Mean Force.
		FY (array of size (nbins[0], nbins[1])): CV1 component of the Mean Force.
		min_grid (array, optional): Lower bound of the simulation domain. Defaults to np.array((-np.pi, -np.pi)).
		min_grid (array, optional): Upper bound of the simulation domain. Defaults to np.array((np.pi, np.pi)).
		nbins (int, optional): number of bins in CV1,CV2. Defaults to 0. When nbins=0, nbins will take the shape of FX.

	Returns:
		X: array of size (nbins[0], nbins[1]) - CV1 grid positions
		Y: array of size (nbins[0], nbins[1]) - CV2 grid positions
		fes: array of size (nbins[0], nbins[1]) - Free Energy Surface
	"""
	if hasattr(nbins, "__len__") == False: nbins = np.shape(FX)        
	
	gridx = np.linspace(min_grid[0], max_grid[0], nbins[0])
	gridy = np.linspace(min_grid[1], max_grid[1], nbins[1])
	grid_spacex = (max_grid[0] - min_grid[0]) / (nbins[0] - 1)
	grid_spacey = (max_grid[1] - min_grid[1]) / (nbins[1] - 1)
	X, Y = np.meshgrid(gridx, gridy)

	# Calculate frequency
	freq_1dx = np.fft.fftfreq(nbins[0], grid_spacex)
	freq_1dy = np.fft.fftfreq(nbins[1], grid_spacey)
	freq_x, freq_y = np.meshgrid(freq_1dx, freq_1dy)
	freq_hypot = np.hypot(freq_x, freq_y)
	freq_sq = np.where(freq_hypot != 0, freq_hypot ** 2, 1E-10)
	# FFTransform and integration
	fourier_x = (np.fft.fft2(FX) * freq_x) / (2 * np.pi * 1j * freq_sq)
	fourier_y = (np.fft.fft2(FY) * freq_y) / (2 * np.pi * 1j * freq_sq)
	# Reverse FFT
	fes_x = np.real(np.fft.ifft2(fourier_x))
	fes_y = np.real(np.fft.ifft2(fourier_y))
	# Construct whole FES
	fes = fes_x + fes_y
	fes = fes - np.min(fes)
	return [X, Y, fes]


In [35]:
HILLS = load_HILLS_2D("../data/HILLS40")
[p_x,p_y] = load_position_2D("../data/position40")

### (0) Naive Method

In [93]:
#Number of evaluations
nhills = 200
n_eval = int(nhills / (stride/10))
[pos_x, pos_y] = [p_x[:stride*n_eval],p_y[:stride*n_eval]]
hills = HILLS[:n_eval]

In [94]:
def MFI_2D(HILLS="HILLS", position_x="position_x", position_y="position_y", bw=1, kT=1,
		   min_grid=np.array((-np.pi, -np.pi)), max_grid=np.array((np.pi, np.pi)), nbins=np.array((200, 200)),
		   log_pace=5, error_pace=10, WellTempered=1, nhills=-1, periodic=1):

	for _init in range(1):
		gridx = np.linspace(min_grid[0], max_grid[0], nbins[0])
		gridy = np.linspace(min_grid[1], max_grid[1], nbins[1])
		grid_space = np.array(((max_grid[0] - min_grid[0]) / (nbins[0]-1), (max_grid[1] - min_grid[1]) / (nbins[1]-1)))
		X, Y = np.meshgrid(gridx, gridy)
		stride = int(len(position_x) / len(HILLS))
		const = (1 / (bw * np.sqrt(2 * np.pi) * stride))

		# Optional - analyse only nhills, if nhills is set
		if nhills > 0:
			total_number_of_hills = nhills
		else:
			total_number_of_hills = len(HILLS)
		bw2 = bw ** 2

		# Initialize force terms
		Fbias_x = np.zeros(nbins)
		Fbias_y = np.zeros(nbins)
		Ftot_num_x = np.zeros(nbins)
		Ftot_num_y = np.zeros(nbins)
		Ftot_den = np.zeros(nbins)
		Ftot_den2 = np.zeros(nbins)
		cutoff=np.zeros(nbins)
		ofv_num_x = np.zeros(nbins)
		ofv_num_y = np.zeros(nbins)
		volume_history = []
		ofe_history = []
		time_history = []

		#Don't Calculate static force in this case
		F_static_x = np.zeros(nbins)
		F_static_y = np.zeros(nbins)

		print("Total no. of Gaussians analysed: " + str(total_number_of_hills))

		# Definition Gamma Factor, allows to switch between WT and regular MetaD
		if WellTempered < 1:
			Gamma_Factor = 1
		else:
			gamma = HILLS[0, 6]
			Gamma_Factor = (gamma - 1) / (gamma)
			
	Ftot_den_limit = 1E-10

	for i in range(total_number_of_hills):
		
		#Probability density limit, below which (fes or error) values aren't considered.
		# Ftot_den_limit = (i+1)*stride * 10**-5
		
		# Build metadynamics potential
		s_x = HILLS[i, 1]  # centre x-position of Gaussian
		s_y = HILLS[i, 2]  # centre y-position of Gaussian
		sigma_meta2_x = HILLS[i, 3] ** 2  # width of Gaussian
		sigma_meta2_y = HILLS[i, 4] ** 2  # width of Gaussian
		height_meta = HILLS[i, 5] * Gamma_Factor  # Height of Gaussian

		periodic_images = find_periodic_point(s_x, s_y, min_grid, max_grid, periodic)
		for j in range(len(periodic_images)):
			kernelmeta = np.exp(-0.5 * (((X - periodic_images[j][0]) ** 2) / sigma_meta2_x + (
						(Y - periodic_images[j][1]) ** 2) / sigma_meta2_y))  # potential erorr in calc. of s-s_t
			Fbias_x = Fbias_x + height_meta * kernelmeta * ((X - periodic_images[j][0]) / sigma_meta2_x);
			Fbias_y = Fbias_y + height_meta * kernelmeta * ((Y - periodic_images[j][1]) / sigma_meta2_y);

		# Estimate the biased proabability density p_t ^ b(s)
		pb_t = np.zeros(nbins)
		Fpbt_x = np.zeros(nbins)
		Fpbt_y = np.zeros(nbins)

		data_x = position_x[i * stride: (i + 1) * stride]
		data_y = position_y[i * stride: (i + 1) * stride]
  
		for j in range(stride):
			periodic_images = find_periodic_point(data_x[j], data_y[j], min_grid, max_grid, periodic)
			for k in range(len(periodic_images)):
				kernel = const * np.exp(
					- (1 / (2 * bw2)) * ((X - periodic_images[k][0]) ** 2 + (Y - periodic_images[k][1]) ** 2))
				pb_t = pb_t + kernel;
				Fpbt_x = Fpbt_x + kernel * kT * (X - periodic_images[k][0]) / bw2
				Fpbt_y = Fpbt_y + kernel * kT * (Y - periodic_images[k][1]) / bw2
		pb_t = np.where(pb_t > Ftot_den_limit, pb_t, 0)  # truncated probability density of window

		# Calculate total probability density
		Ftot_den = Ftot_den + pb_t
		
		# Calculate x-component of Force
		dfds_x = np.divide(Fpbt_x, pb_t, out=np.zeros_like(Fpbt_x), where=pb_t > 0) + Fbias_x - F_static_x
		Ftot_num_x = Ftot_num_x + pb_t * dfds_x
		Ftot_x = np.divide(Ftot_num_x, Ftot_den, out=np.zeros_like(Fpbt_x), where=Ftot_den > 0)
		
		# Calculate y-component of Force
		dfds_y = np.divide(Fpbt_y, pb_t, out=np.zeros_like(Fpbt_y), where=pb_t > 0) + Fbias_y - F_static_y
		Ftot_num_y = Ftot_num_y + pb_t * dfds_y
		Ftot_y = np.divide(Ftot_num_y, Ftot_den, out=np.zeros_like(Fpbt_y), where=Ftot_den > 0)

		# calculate on the fly error components
		Ftot_den2 = Ftot_den2 + pb_t ** 2
		ofv_num_x += pb_t * dfds_x ** 2
		ofv_num_y += pb_t * dfds_y ** 2

		# calculate ofe (standard error)
		[ofv, ofe] = mean_force_variance(Ftot_den, Ftot_den2, Ftot_x, Ftot_y, ofv_num_x, ofv_num_y, Ftot_den_limit=Ftot_den_limit)
		
		ofe_history.append( sum(sum(ofe)) / (np.count_nonzero(ofe)))
		time_history.append(HILLS[i,0])
		#print progress
		if (i + 1) % log_pace == 0:
			print("|" + str(i + 1) + "/" + str(total_number_of_hills) + "|==> Average Mean Force Error: " + str(ofe_history[-1]))
			
	return [X, Y, Ftot_den, Ftot_den2, Ftot_x, Ftot_y, ofv_num_x, ofv_num_y, ofv, ofe, cutoff, volume_history, ofe_history, time_history]


In [99]:
start = time.time()

[X, Y, Ftot_den, Ftot_den2, Ftot_x, Ftot_y, ofv_num_x, ofv_num_y, ofv, ofe, cutoff, volume_history, ofe_history, time_history] = MFI_2D(HILLS=hills, position_x=pos_x, position_y=pos_y, bw=0.1, kT=kT,
		   min_grid=min_grid, max_grid=max_grid, nbins=nbins,
		   log_pace=n_eval, error_pace=n_eval/10, nhills=n_eval, periodic=1)


t_normal = time.time()-start	
print("time for >>MFI_normal<< is:", t_normal)

[X, Y, FES] = FFT_intg_2D(Ftot_x, Ftot_y)

Total no. of Gaussians analysed: 200


<ipython-input-94-27c832a61c41>:107: RuntimeWarning: invalid value encountered in double_scalars
  ofe_history.append( sum(sum(ofe)) / (np.count_nonzero(ofe)))


|200/200|==> Average Mean Force Error: 37.044953421952854
time for >>MFI_normal<< is: 6.723116159439087


### (1) Numpy method (optimise the use of numpy)

In [96]:
def mean_force_variance_numpy(Ftot_den, Ftot_den2, Ftot_x, Ftot_y, ofv_num_x, ofv_num_y, Ftot_den_limit=0):

	Ftot_den_sq = np.square(Ftot_den)
	Ftot_den_diff = Ftot_den_sq-Ftot_den2
	bessel_corr = np.divide(Ftot_den_sq , Ftot_den_diff, out=np.zeros_like(Ftot_den), where=Ftot_den_diff > 0)

	ofv_x = np.multiply(np.divide(ofv_num_x , Ftot_den, out=np.zeros_like(Ftot_den), where=Ftot_den > 0) - np.square(Ftot_x) , bessel_corr )
	ofe_x = np.sqrt(ofv_x)
	
	ofv_y = np.multiply(np.divide(ofv_num_y , Ftot_den, out=np.zeros_like(Ftot_den), where=Ftot_den > 0) - np.square(Ftot_y) , bessel_corr )
	ofe_y = np.sqrt(ofv_y)
	
	ofv = np.sqrt(np.square(ofv_x) + np.square(ofv_y))
	ofe = np.sqrt(np.square(ofe_x) + np.square(ofe_y))
		
	return [ofv, ofe]

def MFI_2D_numpy(HILLS="HILLS", position_x="position_x", position_y="position_y", bw=1, kT=1,
		   min_grid=np.array((-np.pi, -np.pi)), max_grid=np.array((np.pi, np.pi)), nbins=np.array((200, 200)),
		   log_pace=5, error_pace=10, WellTempered=1, nhills=-1, periodic=1):

	for _init in range(1):
		gridx = np.linspace(min_grid[0], max_grid[0], nbins[0])
		gridy = np.linspace(min_grid[1], max_grid[1], nbins[1])
		grid_space = np.array(((max_grid[0] - min_grid[0]) / (nbins[0]-1), (max_grid[1] - min_grid[1]) / (nbins[1]-1)))
		X, Y = np.meshgrid(gridx, gridy)
		stride = int(len(position_x) / len(HILLS))
		const = (1 / (bw * np.sqrt(2 * np.pi) * stride))

		# Optional - analyse only nhills, if nhills is set
		if nhills > 0:
			total_number_of_hills = nhills
		else:
			total_number_of_hills = len(HILLS)
		bw2 = bw ** 2

		# Initialize force terms
		Fbias_x = np.zeros(nbins)
		Fbias_y = np.zeros(nbins)
		Ftot_num_x = np.zeros(nbins)
		Ftot_num_y = np.zeros(nbins)
		Ftot_den = np.zeros(nbins)
		Ftot_den2 = np.zeros(nbins)
		cutoff=np.zeros(nbins)
		ofv_num_x = np.zeros(nbins)
		ofv_num_y = np.zeros(nbins)
		volume_history = []
		ofe_history = []
		time_history = []

		#Don't Calculate static force in this case
		F_static_x = np.zeros(nbins)
		F_static_y = np.zeros(nbins)

		print("Total no. of Gaussians analysed: " + str(total_number_of_hills))

		# Definition Gamma Factor, allows to switch between WT and regular MetaD
		if WellTempered < 1:
			Gamma_Factor = 1
		else:
			gamma = HILLS[0, 6]
			Gamma_Factor = (gamma - 1) / (gamma)
			
		Ftot_den_limit = 1E-10

	for i in range(total_number_of_hills):
				
		# Build metadynamics potential
		s_x = HILLS[i, 1]  # centre x-position of Gaussian
		s_y = HILLS[i, 2]  # centre y-position of Gaussian
		sigma_meta2_x = HILLS[i, 3] ** 2  # width of Gaussian
		sigma_meta2_y = HILLS[i, 4] ** 2  # width of Gaussian
		height_meta = HILLS[i, 5] * Gamma_Factor  # Height of Gaussian

		periodic_images = find_periodic_point(s_x, s_y, min_grid, max_grid, periodic)
		for j in range(len(periodic_images)):

			kernelmeta_x = np.exp( - np.square(gridx - periodic_images[j][0]) / (2 * sigma_meta2_x)) * height_meta
			kernelmeta_y = np.exp( - np.square(gridy - periodic_images[j][1]) / (2 * sigma_meta2_y))
			# kernelmeta = np.outer(kernelmeta_y, kernelmeta_x)
      
			Fbias_x = Fbias_x + np.outer(kernelmeta_y, np.multiply(kernelmeta_x, (gridx - periodic_images[j][0])) / sigma_meta2_x )
			Fbias_y = Fbias_y + np.outer(np.multiply(kernelmeta_y, (gridy - periodic_images[j][1])) / sigma_meta2_y, kernelmeta_x )

		# Estimate the biased proabability density p_t ^ b(s)
		pb_t = np.zeros(nbins)
		Fpbt_x = np.zeros(nbins)
		Fpbt_y = np.zeros(nbins)

		data_x = position_x[i * stride: (i + 1) * stride]
		data_y = position_y[i * stride: (i + 1) * stride]
  
		for j in range(stride):
			periodic_images = find_periodic_point(data_x[j], data_y[j], min_grid, max_grid, periodic)
			for k in range(len(periodic_images)):

				kernel_x = np.exp( - np.square(gridx - periodic_images[k][0]) / (2 * bw2)) * const #add constant here for less computations
				kernel_y = np.exp( - np.square(gridy - periodic_images[k][1]) / (2 * bw2))
				kernel = np.outer(kernel_y, kernel_x)
    
				kernel_x = kernel_x * kT / bw2 #add constant here for less computations
    
				pb_t = pb_t + kernel
    
				Fpbt_x = Fpbt_x + np.outer(kernel_y, np.multiply(kernel_x, (gridx - periodic_images[k][0])) )
				Fpbt_y = Fpbt_y + np.outer(np.multiply(kernel_y, (gridy - periodic_images[k][1])) , kernel_x )

		pb_t = np.where(pb_t > Ftot_den_limit, pb_t, 0)  # truncated probability density of window

		# Calculate total probability density
		Ftot_den = Ftot_den + pb_t
		
		# Calculate x-component of Force
		dfds_x = np.divide(Fpbt_x, pb_t, out=np.zeros_like(Ftot_den), where=pb_t > 0) + Fbias_x - F_static_x
		Ftot_num_x = Ftot_num_x + np.multiply(pb_t, dfds_x)
		
		# Calculate y-component of Force
		dfds_y = np.divide(Fpbt_y, pb_t, out=np.zeros_like(Ftot_den), where=pb_t > 0) + Fbias_y - F_static_y
		Ftot_num_y = Ftot_num_y + np.multiply(pb_t, dfds_y)

		# calculate on the fly error components
		Ftot_den2 += np.square(pb_t)
		ofv_num_x += np.multiply(pb_t, np.square(dfds_x))
		ofv_num_y += np.multiply(pb_t, np.square(dfds_y))

		#force and calculate error
		if (i + 1) % error_pace == 0 or (i+1) == total_number_of_hills:
			
			Ftot_x = np.divide(Ftot_num_x, Ftot_den, out=np.zeros_like(Ftot_den), where=Ftot_den > 0)
			Ftot_y = np.divide(Ftot_num_y, Ftot_den, out=np.zeros_like(Ftot_den), where=Ftot_den > 0)

			[ofv, ofe] = mean_force_variance_numpy(Ftot_den, Ftot_den2, Ftot_x, Ftot_y, ofv_num_x, ofv_num_y)
		
			ofe_history.append( np.sum(ofe) / (np.count_nonzero(ofe)))
			time_history.append(HILLS[i,0])
		
  		#print progress
		if (i + 1) % log_pace == 0:
			print("|" + str(i + 1) + "/" + str(total_number_of_hills) + "|==> Average Mean Force Error: " + str(ofe_history[-1]))
			
	return [X, Y, Ftot_den, Ftot_den2, Ftot_x, Ftot_y, ofv_num_x, ofv_num_y, ofv, ofe, cutoff, volume_history, ofe_history, time_history]


In [100]:
start = time.time()

[X, Y, Ftot_dens, Ftot_den2s, Ftot_xs, Ftot_ys, ofv_num_xs, ofv_num_ys, ofv, ofe, cutoff, volume_history, ofe_history, time_history] = MFI_2D_numpy(HILLS=HILLS, position_x=p_x, position_y=p_y, bw=0.1, kT=kT,
		   min_grid=min_grid, max_grid=max_grid, nbins=nbins,
		   log_pace=n_eval, error_pace=n_eval/10, nhills=n_eval, periodic=1)


t_numpy = time.time()-start
print("\ntime for >>MFI_numpy<< is:", t_numpy)
print("time saving:", round(100*(t_normal - t_numpy)/t_normal, 2), "%")

[X, Y, FESs] = FFT_intg_2D(Ftot_xs, Ftot_ys)
	
print("\nERROR:")
print("differecen in Ftot_den:", sum(sum(abs(Ftot_den - Ftot_dens))) / 200**2)
print("differecen in Ftot_den2:", sum(sum(abs(Ftot_den2 - Ftot_den2s))) / 200**2)
print("differecen in Ftot_x:", sum(sum(abs(Ftot_x - Ftot_xs))) / 200**2)
print("differecen in Ftot_y:", sum(sum(abs(Ftot_y - Ftot_ys))) / 200**2)
print("differecen in ofv_num_x:", sum(sum(abs(ofv_num_x - ofv_num_xs))) / 200**2)
print("differecen in ofv_num_y:", sum(sum(abs(ofv_num_y - ofv_num_ys))) / 200**2)
print("\ndifferecen in FES:", sum(sum(abs(FES-FESs))) / 200**2)


Total no. of Gaussians analysed: 200
|200/200|==> Average Mean Force Error: 37.04495342195384

time for >>MFI_numpy<< is: 1.766749620437622
time saving: 73.72 %

ERROR:
differecen in Ftot_den: 9.36287822348276e-17
differecen in Ftot_den2: 4.2760839957035474e-17
differecen in Ftot_x: 2.654507001579112e-15
differecen in Ftot_y: 2.2315611815024172e-15
differecen in ofv_num_x: 9.65193627643362e-14
differecen in ofv_num_y: 8.353243712830433e-14

differecen in FES: 7.261746759468224e-15


### (2) Numba method

### (3) No-Loop method

### Plot differences

In [60]:
# plt.figure(1)
# plt.contourf(X,Y,FESs)
# plt.colorbar()
# plt.scatter(HILLS[:n_eval,1], HILLS[:n_eval,2], c="r", s=2)

# plt.figure(4)
# plt.contourf(X,Y,FES)
# plt.colorbar()
# plt.scatter(HILLS[:n_eval,1], HILLS[:n_eval,2], c="r", s=2)


# # plt.figure(2)
# # plt.contourf(X,Y,Ftot_xs, levels = np.linspace(-160, 160, 21))
# # plt.colorbar()
# # plt.scatter(HILLS[:n_eval,1], HILLS[:n_eval,2], c="r", s=2)

# # plt.figure(5)
# # plt.contourf(X,Y,Ftot_x, levels = np.linspace(-160, 160, 21))
# # plt.colorbar()
# # plt.scatter(HILLS[:n_eval,1], HILLS[:n_eval,2], c="r", s=2)

# plt.figure(6)
# plt.contourf(X,Y,Ftot_x-Ftot_xs)
# plt.colorbar()
# plt.scatter(HILLS[:n_eval,1], HILLS[:n_eval,2], c="r", s=2)

# plt.show()